In [ ]:
!pip install -q transformers datasets accelerate

import os
from datasets import Dataset

# ========= 1) RUTAS =========
POS_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/positivos"
NEG_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/negativos_gruesos"
NEGF_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/negativos_finos"


In [ ]:
# ========= 2) CARGA DE TXT =========
def cargar_textos(carpeta, etiqueta):
    ejemplos = []
    for fname in os.listdir(carpeta):
        if fname.endswith(".txt"):
            ruta = os.path.join(carpeta, fname)
            with open(ruta, "r", encoding="utf-8") as f:
                texto = f.read()
            ejemplos.append({"text": texto, "label": etiqueta})
    return ejemplos

positivos = cargar_textos(POS_DIR, 1)
negativos = cargar_textos(NEG_DIR, 0)
finos = cargar_textos(NEGF_DIR, 0)
negativos = negativos[len(finos):]
negativos.extend(finos)
datos = positivos + negativos

print("Total train =", len(datos))

dataset = Dataset.from_list(datos)

Total train = 1210


In [ ]:
# ========= 3) TOKENIZACIÓN =========
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
MAX_LENGTH = 512  # ajustable

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.remove_columns(["text"])
tokenized.set_format("torch")

Map:   0%|          | 0/1210 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
# ========= 4) MODELO Y TRAINER =========
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/StoryWriter/Clasificador/Grueso+fino",
    learning_rate=2e-5,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2166921709.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.345400
200,0.062400
300,0.034200
400,0.038900
500,0.055600
600,0.023400


TrainOutput(global_step=608, training_loss=0.09209272734159113, metrics={'train_runtime': 207.7959, 'train_samples_per_second': 23.292, 'train_steps_per_second': 2.926, 'total_flos': 1273457507942400.0, 'train_loss': 0.09209272734159113, 'epoch': 4.0})

In [ ]:
# ========= 5) GUARDAR MODELO =========
save_dir = "/content/drive/MyDrive/StoryWriter/Clasificador/Grueso+fino"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Modelo guardado en:", save_dir)


Modelo guardado en: /content/drive/MyDrive/StoryWriter/Clasificador/Grueso+fino
